# CROP Arima model
This notebook checks outputs of the Arima model

In [91]:
#!pip3 install psycopg2
#!pip3 install plotly

In [92]:
import os
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

Parameters

In [93]:
crop_host = "cropapptestsqlserver.postgres.database.azure.com"
crop_port = "5432"
crop_dbname = "app_db"
crop_user = "cropdbadmin@cropapptestsqlserver"
crop_password = "QhXZ7qZddDr224Mc2P4k"

In [94]:
conn = psycopg2.connect(
    host=crop_host, port=crop_port, dbname=crop_dbname, 
    user=crop_user, password=crop_password)

cur = conn.cursor()

### Arima Data Access

All Arima data for 3 sensors

In [95]:
#dt_to = datetime.now()
dt_to = datetime(2021, 4, 27)  
dt_from = dt_to + timedelta(days=-15)

In [96]:
#sql_command = """SELECT * FROM model_run"""

sql_command = """SELECT model.id, model.model_name,  model_run.sensor_id, model_run.time_forecast, model_measure.measure_name, model_product.run_id, model_value.prediction_index,model_value.prediction_value 
FROM model, model_run, model_measure, model_product, model_value WHERE model.id =1 AND model_run.model_id = model.id  
AND model_product.run_id = model_run.id AND model_product.measure_id = model_measure.id AND model_value.product_id = model_product.id
AND model_run.time_forecast >= '%s' AND model_run.time_forecast < '%s'""" % (dt_from, dt_to)


In [97]:
arima_raw = pd.read_sql(sql_command, conn)
arima = arima_raw.drop_duplicates()
arima

,id,model_name,sensor_id,time_forecast,measure_name,run_id,prediction_index,prediction_value
0,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,1,19.417568
1,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,2,19.529451
2,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,3,19.816390
3,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,4,20.045238
4,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,5,20.172127
...,...,...,...,...,...,...,...,...
139,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,44,12.821713
140,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,45,12.659180
141,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,46,12.005555
142,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,47,12.126255


## Calculate and add column with prediciton time

In [98]:
prediction_hours= []
for i in range(len(arima)):
   prediction_hours.append(arima['time_forecast'][i] + timedelta(hours= int(arima['prediction_index'][i])))

arima['prediction_time'] = prediction_hours

arima


,id,model_name,sensor_id,time_forecast,measure_name,run_id,prediction_index,prediction_value,prediction_time
0,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,1,19.417568,2021-04-26 17:00:00
1,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,2,19.529451,2021-04-26 18:00:00
2,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,3,19.816390,2021-04-26 19:00:00
3,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,4,20.045238,2021-04-26 20:00:00
4,1,arima,18,2021-04-26 16:00:00,Mean Temperature (Degree Celcius),107,5,20.172127,2021-04-26 21:00:00
...,...,...,...,...,...,...,...,...,...
139,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,44,12.821713,2021-04-28 12:00:00
140,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,45,12.659180,2021-04-28 13:00:00
141,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,46,12.005555,2021-04-28 14:00:00
142,1,arima,18,2021-04-26 16:00:00,Lower Bound Temperature (Degree Celcius),107,47,12.126255,2021-04-28 15:00:00


In [99]:
#test = arima[arima['run_id'] ==  86]


In [100]:
#unique_run_ids = arima['run_id'].unique()
unique_sensors = arima['sensor_id'].unique()
unique_time_forecast = arima['time_forecast'].unique()
unique_measures = arima['measure_name'].unique()

unique_time_forecast

array(['2021-04-26T16:00:00.000000000'], dtype='datetime64[ns]')

In [101]:
date_time = pd.to_datetime(unique_time_forecast[0])
date_time
#type(date_time)
#type(datetime(2021, 6, 16))

Timestamp('2021-04-26 16:00:00')

### Get Temperature data from Zensie Sensors

All Zensie sensors

In [102]:
dt_to_z = date_time + timedelta(days=+3) #datetime(2021, 6, 16)
dt_from_z = dt_to_z + timedelta(days=-5)

In [103]:
sql_command_zensie = """SELECT sensors.name, zensie_trh_data.* FROM sensor_types, sensors, zensie_trh_data WHERE sensors.id = zensie_trh_data.sensor_id AND zensie_trh_data.timestamp >= '%s' AND zensie_trh_data.timestamp < '%s'""" % (dt_from_z, dt_to_z)

In [104]:
zensie_raw = pd.read_sql(sql_command_zensie, conn)
zensie_df = zensie_raw.drop_duplicates()
zensie_df

,name,id,sensor_id,timestamp,temperature,humidity,time_created,time_updated
0,FARM_T/RH_16B1,497754,18,2021-04-24 16:05:00,18.250000,77.0,2021-04-24 17:00:04.568520,None
5,FARM_T/RH_16B1,497755,18,2021-04-24 16:15:00,18.500000,77.0,2021-04-24 17:00:04.568520,None
10,FARM_T/RH_16B1,497756,18,2021-04-24 16:25:00,18.600000,77.0,2021-04-24 17:00:04.568520,None
15,FARM_T/RH_16B1,497757,18,2021-04-24 16:35:00,18.650000,77.0,2021-04-24 17:00:04.568520,None
20,TUNNEL7_T/RH_1,497760,19,2021-04-24 16:05:00,18.500000,83.0,2021-04-24 17:00:06.787226,None
...,...,...,...,...,...,...,...,...
31765,R&D_T/RH_2,504145,25,2021-04-29 15:50:00,20.250000,48.0,2021-04-29 17:00:17.827567,None
31770,Farm_T/RH_16B2,504150,27,2021-04-29 15:45:00,21.000000,62.0,2021-04-29 17:00:19.858794,None
31775,Farm_T/RH_16B2,504151,27,2021-04-29 15:55:00,20.500000,62.0,2021-04-29 17:00:19.858794,None
31780,R&D_T/RH_1,504156,24,2021-04-29 15:40:00,18.400000,45.0,2021-04-29 17:00:22.015056,None


## Visualisation and parameters

In [105]:
time_forecast_id= 0
time_forecast = unique_time_forecast[time_forecast_id]

In [106]:
color = ["red", 'white', 'white']
for j in range (len(unique_sensors)):

    # set up plotly figure
    fig = go.Figure()
    
    arima_df = arima[(arima['sensor_id'] == unique_sensors[j]) & (arima['time_forecast'] == time_forecast)]

    # add line / trace 1 to figure
    for i in range(len (unique_measures)):
        trace_sensor = arima_df[arima_df['measure_name'] == unique_measures[i]]
        fig.add_trace(go.Scatter(
            x=trace_sensor['prediction_time'],
            y=trace_sensor['prediction_value'],
            #hovertext=df['A_info'],
            #hoverinfo="text",
            name=unique_measures[i],#unique_measures[j],
            marker=dict(
                color=color[i],
            ),
            showlegend=True
        ))

    # add line / trace 2 to figure
    
    zensie_df_s = zensie_df[zensie_df['sensor_id'] == unique_sensors[j]]
    fig.add_trace(go.Scatter(
        x=zensie_df_s['timestamp'],
        y=zensie_df_s['temperature'],
        name='zensie',
        marker=dict(
            color="green"
        ),
        showlegend=False
    ))
    
    fig.update_layout(
    title="Arima and Zensie data: Sensor id %s" % unique_sensors[j] ,
    xaxis_title="Date",
    yaxis_title="Temperature")

    fig.show()
    
    customAnnotations(df=trace_sensor, xStart = '2020-08-04', xEnd = '2020-08-06', yVal='regression_sales')

## Calculate Square root mean error

In [124]:
import sklearn
from sklearn.metrics import mean_squared_error
import bisect

In [108]:
unique_measures

array(['Mean Temperature (Degree Celcius)',
       'Upper Bound Temperature (Degree Celcius)',
       'Lower Bound Temperature (Degree Celcius)'], dtype=object)

In [118]:
arima['prediction_time']

0     2021-04-26 17:00:00
1     2021-04-26 18:00:00
2     2021-04-26 19:00:00
3     2021-04-26 20:00:00
4     2021-04-26 21:00:00
              ...        
139   2021-04-28 12:00:00
140   2021-04-28 13:00:00
141   2021-04-28 14:00:00
142   2021-04-28 15:00:00
143   2021-04-28 16:00:00
Name: prediction_time, Length: 144, dtype: datetime64[ns]

In [192]:
zensie_f = zensie_df[['timestamp','temperature']]


In [200]:
f= zensie_f.groupby(by=[zensie_f.timestamp.map(lambda x: "%04d-%02d-%02d-%02d" % (x.year, x.month, x.day, x.hour)), "temperature"]).sum()


                 

f

Empty DataFrame
Columns: []
Index: [(2021-04-24-16, 16.2000007629395), (2021-04-24-16, 16.25), (2021-04-24-16, 16.2999992370605), (2021-04-24-16, 18.25), (2021-04-24-16, 18.3999996185303), (2021-04-24-16, 18.5), (2021-04-24-16, 18.5499992370605), (2021-04-24-16, 18.6000003814697), (2021-04-24-16, 18.6499996185303), (2021-04-24-16, 18.7000007629395), (2021-04-24-16, 18.75), (2021-04-24-16, 18.7999992370605), (2021-04-24-16, 19.25), (2021-04-24-16, 19.4500007629395), (2021-04-24-16, 19.5499992370605), (2021-04-24-16, 19.6000003814697), (2021-04-24-16, 19.6499996185303), (2021-04-24-16, 19.8999996185303), (2021-04-24-16, 20.1499996185303), (2021-04-24-16, 20.25), (2021-04-24-16, 20.3999996185303), (2021-04-24-16, 20.6499996185303), (2021-04-24-16, 20.7999992370605), (2021-04-24-16, 20.8500003814697), (2021-04-24-16, 20.8999996185303), (2021-04-24-16, 20.9500007629395), (2021-04-24-16, 21.0499992370605), (2021-04-24-16, 21.1000003814697), (2021-04-24-16, 21.25), (2021-04-24-16, 21.2999992370605), (2021-04-24-16, 21.3999996185303), (2021-04-24-16, 21.5499992370605), (2021-04-24-16, 21.6499996185303), (2021-04-24-16, 23.7000007629395), (2021-04-24-16, 23.75), (2021-04-24-16, 23.7999992370605), (2021-04-24-17, 16.3500003814697), (2021-04-24-17, 16.3999996185303), (2021-04-24-17, 16.4500007629395), (2021-04-24-17, 17.9500007629395), (2021-04-24-17, 18.1000003814697), (2021-04-24-17, 18.25), (2021-04-24-17, 18.3500003814697), (2021-04-24-17, 18.5), (2021-04-24-17, 18.6000003814697), (2021-04-24-17, 18.7999992370605), (2021-04-24-17, 18.8500003814697), (2021-04-24-17, 18.8999996185303), (2021-04-24-17, 19.25), (2021-04-24-17, 19.2999992370605), (2021-04-24-17, 19.3500003814697), (2021-04-24-17, 19.5), (2021-04-24-17, 19.6000003814697), (2021-04-24-17, 20.0), (2021-04-24-17, 20.0499992370605), (2021-04-24-17, 20.1000003814697), (2021-04-24-17, 20.2000007629395), (2021-04-24-17, 20.6000003814697), (2021-04-24-17, 20.6499996185303), (2021-04-24-17, 20.7000007629395), (2021-04-24-17, 20.75), (2021-04-24-17, 20.9500007629395), (2021-04-24-17, 21.6499996185303), (2021-04-24-17, 21.7000007629395), (2021-04-24-17, 21.75), (2021-04-24-17, 23.7999992370605), (2021-04-24-17, 23.8500003814697), (2021-04-24-17, 23.8999996185303), (2021-04-24-18, 16.5), (2021-04-24-18, 16.5499992370605), (2021-04-24-18, 18.6499996185303), (2021-04-24-18, 18.7000007629395), (2021-04-24-18, 18.75), (2021-04-24-18, 18.7999992370605), (2021-04-24-18, 18.8999996185303), (2021-04-24-18, 18.9500007629395), (2021-04-24-18, 19.0), (2021-04-24-18, 19.0499992370605), (2021-04-24-18, 19.7000007629395), (2021-04-24-18, 19.75), (2021-04-24-18, 19.8500003814697), (2021-04-24-18, 19.8999996185303), (2021-04-24-18, 20.0), (2021-04-24-18, 20.1000003814697), (2021-04-24-18, 20.25), (2021-04-24-18, 20.3500003814697), (2021-04-24-18, 20.3999996185303), (2021-04-24-18, 20.4500007629395), (2021-04-24-18, 20.5), (2021-04-24-18, 20.5499992370605), (2021-04-24-18, 20.6000003814697), (2021-04-24-18, 20.6499996185303), (2021-04-24-18, 20.75), (2021-04-24-18, 20.9500007629395), (2021-04-24-18, 23.8500003814697), (2021-04-24-18, 23.8999996185303), (2021-04-24-19, 16.5499992370605), (2021-04-24-19, 16.6000003814697), (2021-04-24-19, 18.8500003814697), (2021-04-24-19, 18.8999996185303), ...]

[3724 rows x 0 columns]

In [140]:
myPairs = list(zensie_f.iteritems())
myPairs.sort()

import bisect
i = bisect.bisect_left(myPairs, arima['prediction_time'])


#i = bisect.bisect_left(myPairs, arima['prediction_time'][0])

ValueError: Lengths must match

In [109]:
actual = zensie_df['temperature']
predicted = arima_df [(arima['measure_name'] == unique_measures[0])] 

mse = sklearn.metrics.mean_squared_error(actual, predicted)

rmse = math.sqrt(mse)

ValueError: Found input variables with inconsistent numbers of samples: [6358, 48]